In [2]:
import os
os.chdir("/content/drive/MyDrive/çalışmalarım/derin öğrenme kursu/titanic")
!pwd

/content/drive/MyDrive/çalışmalarım/derin öğrenme kursu/titanic


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')



In [4]:
# on islemler
def preprocess(data):
  #null degerleri doldurur
  data.Cabin.fillna('0', inplace=True)
  #kelime bilgilerini sayi bilgilerine cevirme
  #kabinleri numaraladirma
  data.loc[data.Cabin.str[0]=='A','Cabin']=1
  data.loc[data.Cabin.str[0]=='B','Cabin']=2
  data.loc[data.Cabin.str[0]=='C','Cabin']=3
  data.loc[data.Cabin.str[0]=='D','Cabin']=4
  data.loc[data.Cabin.str[0]=='E','Cabin']=5
  data.loc[data.Cabin.str[0]=='F','Cabin']=6
  data.loc[data.Cabin.str[0]=='G','Cabin']=7
  data.loc[data.Cabin.str[0]=='T','Cabin']=8
  #cinsiyetleri numaralandirma
  data['Sex'].replace('female',1,inplace=True)
  data['Sex'].replace('male',2,inplace=True)
  #gemiye binis limanlarini numaralandirma
  data['Embarked'].replace('S',1,inplace=True)
  data['Embarked'].replace('C',2,inplace=True)
  data['Embarked'].replace('Q',3,inplace=True)
  #yas bilgisi olmayana medyan yas atamasi (genel ortalama bozulmayacak)
  data['Age'].fillna(data['Age'].median(),inplace=True)
  #ucret
  data['Fare'].fillna(data['Fare'].median(),inplace=True)
  #sehir
  data['Embarked'].fillna(data['Embarked'].median(),inplace=True)

  #bos olan degerleri silmek isteseydim
  #data.dropna(subset=['Fare', 'Embarked'],inplace=True,how='any')
  
  return data

In [5]:
def group_titles(data):
  #isimler ve genel basliklar
  data['Names']=data['Name'].map(lambda x: len(re.split(' ',x)))
  data['Title']=data['Name'].map(lambda x: re.search(', (.+?) ',x).group(1))
  data['Title'].replace('Master.',0,inplace=True)
  data['Title'].replace('Mr.',1,inplace=True)
  data['Title'].replace(['Ms.','Mlle.','Miss.'],2,inplace=True)
  data['Title'].replace(['Mme.','Mrs.'],3,inplace=True)
  data['Title'].replace(['Dona.','Lady.','the Countess.','Capt.','Col.','Don.','Dr.','Major.','Rev.','Sir.','Jonkheer.','the'],4,inplace=True)

In [6]:
def data_subset(data):
  #ozniteliklerden olusan veri tabani ve oznitelik sayisi dondurur
  features=['Pclass','SibSp','Parch','Sex','Names','Title','Age','Cabin']
  lenght_features=len(features)
  subset=data[features]
  return subset, lenght_features

In [7]:
#Model tasarimi
def create_model(train_set_size, input_lenght, num_epochs, batch_size):
  #sinir agi modelinin nasil olacagi belirlenir
  model=Sequential()
  #Dense baglanti katman
  model.add(Dense(7,input_dim=input_lenght, activation='softplus'))
  model.add(Dense(3,activation='softplus'))
  model.add(Dense(1, activation='softplus'))
  #learning rate- ogrenme orani
  lr=.001
  adam0=Adam(lr=lr)
  #modeli derleme ve model agirliklarini kaydetme
  #loss function= binary crossentropy
  model.compile(loss='binary_crossentropy',optimizer=adam0, metrics=['accuracy'])
  filepath='weights.best.hdf5'
  #en iyi sonuclari kaydetme mode=max
  checkpoint=ModelCheckpoint(filepath, monitory='acc', verbose=1, save_best_only=True,mode='max')
  callbacks_list=[checkpoint]

  history_model=model.fit(X_train[:train_set_size],Y_train[:train_set_size],callbacks=callbacks_list,epochs=num_epochs,batch_size=batch_size,verbose=0)
  return model, history_model

In [11]:
#grafik
def plots(history):
  #history'deki sonuclari gosterme
  loss_history=history.history['loss']
  acc_history=history.history['accuracy']
  epochs=[(i+1) for i in range(num_epochs)]

  ax=plt.subplot(211)
  ax.plot(epochs,loss_history,color='red')
  ax.set_xlabel('Epochs')
  ax.set_ylabel('Error Rate per Epoch\n')

  ax2=plt.subplot(212)
  ax2.plot(epochs,acc_history,color='blue')
  ax.set_xlabel('Epochs')
  ax.set_ylabel('Accuracy per Epoch\n')

  plt.subplots_adjust(hspace=0.8)
  plt.savefig('Accuracy_loss.png')
  plt.close()

In [9]:
def test(batch_size):
  test=pd.read_csv('test.csv',header=0)
  test_ids=test['PassengerId']
  test = preprocess(test)
  group_titles(test)
  testdata, _ =data_subset(test)
  
  X_test=np.array(testdata).astype(float)

  output=model.predict(X_test,batch_size=batch_size,verbose=0)
  output=output.reshape((418,))
  #kisilerin idleriyle hayatta olup olmama bilgileri birlestirilir
  column_1=np.concatenate((['PassengerId'],test_ids),axis=0)
  column_2=np.concatenate((['Survived'],output),axis=0)

  f=open("output.csv","w")
  writer=csv.writer(f)
  for i in range(len(column_1)):
    writer.writerow([column_1[i]]+ [column_2[i]])
  f.close()

In [12]:
#sonuclarin yeniden uretilebilir olmasi icin
seed=7
np.random.seed(seed)
#train seti okunup hazirlanir
train=pd.read_csv('train.csv',header=0)
preprocess(train)
group_titles(train)
#epoch sayisi belirlenir
num_epochs=100
#her iteraasyonda alacagi küme sayisi
batch_size=32

traindata, lenght_features=data_subset(train)

Y_train=np.array(train['Survived']).astype(int)
X_train=np.array(traindata).astype(float)
#egitim ve dogrulama kume orani belirlenir
train_set_size=int(.67*len(X_train))
#model ve model gecmisi olusturulur
model, history_model=create_model(train_set_size,lenght_features,num_epochs,batch_size)

plots(history_model)
#train->train+validation
X_validation=X_train[train_set_size:]
Y_validation=Y_train[train_set_size:]

loss_and_metrics=model.evaluate(X_validation,Y_validation,batch_size=batch_size)
print(loss_and_metrics)

test(batch_size)

10/10 [==============================] - 0s 3ms/step - loss: 0.4661 - accuracy: 0.8203
[0.4661064147949219, 0.8203389644622803]
